In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.customer_details_bronze
AS SELECT *, current_timestamp() as load_time
FROM cloud_files('s3a://databricks-p7865/ecommerce-data-files/csv/Customer_details/',  "csv", map("cloudFiles.inferColumnTypes", "true"))

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.seller_details_bronze
AS SELECT *, current_timestamp() as load_time
FROM cloud_files('s3://databricks-p7865/ecommerce-data-files/csv/seller_details/',  "csv", map("cloudFiles.inferColumnTypes", "true"))

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.product_details_bronze
AS SELECT *, current_timestamp() as load_time
FROM cloud_files('s3://databricks-p7865/ecommerce-data-files/csv/product_details/',  "csv", map("cloudFiles.inferColumnTypes", "true"))

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE ecommerce_dashboard.tables.sales_bronze
AS SELECT *, current_timestamp() as load_time
FROM cloud_files('s3://databricks-p7865/ecommerce-data-files/csv/sales/',  "csv", map("cloudFiles.inferColumnTypes", "true"))

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.customer_details_silver as
select 
customer_id,
customer_name,
region,
country

 from stream(LIVE.ecommerce_dashboard.tables.customer_details_bronze)
 where customer_id is not null

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.product_details_silver as
select 
product_id,
product_name,
category,
price
 from stream(LIVE.ecommerce_dashboard.tables.product_details_bronze) where price >0 and product_id is not null

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.seller_details_silver as
select 
seller_id,
seller_name,
region,
country

 from stream(LIVE.ecommerce_dashboard.tables.seller_details_bronze) where seller_id is not null

In [0]:
CREATE OR REFRESH STREAMING live TABLE ecommerce_dashboard.tables.sales_silver as
 
 select cte.*  , p.price*cte.quantity as total_price from (select 
sale_id,
sale_date,
customer_id,
seller_id,
product_id,
quantity
 from  stream(LIVE.ecommerce_dashboard.tables.sales_bronze) 
 where quantity >0 and sale_id is not null ) as cte 
  left join
 LIVE.ecommerce_dashboard.tables.product_details_silver p 
 on cte.product_id=p.product_id

In [0]:
CREATE OR REFRESH LIVE TABLE ecommerce_dashboard.tables.sales
AS
SELECT
    c.region,
    c.country,
    p.category,
    p.product_name,
    se.seller_name,
    s.sale_date,
    SUM(s.quantity)    AS quantity,
    SUM(s.total_price) AS total_sales
FROM LIVE.ecommerce_dashboard.tables.sales_silver s
INNER JOIN LIVE.ecommerce_dashboard.tables.customer_details_silver c
    ON s.customer_id = c.customer_id
INNER JOIN LIVE.ecommerce_dashboard.tables.product_details_silver p
    ON s.product_id = p.product_id
INNER JOIN LIVE.ecommerce_dashboard.tables.seller_details_silver se
    ON s.seller_id = se.seller_id
GROUP BY
    c.region,
    c.country,
    p.category,
    p.product_name,
    s.sale_date,
    se.seller_name;
